In [4]:
from util import load_pickle
from a2_spanbert import load_csv, load_entry

In [5]:
groupedby_n2c2 = load_pickle("groupedby_n2c2.pickle")

In [6]:
groupedby_n2c2

{164366: {45: {'ADR': ['rash'], 'DRUG': []},
  19: {'ADR': ['gastritis'], 'DRUG': []},
  16: {'ADR': ['dyslipidemia'], 'DRUG': []},
  18: {'ADR': ['low back pain'], 'DRUG': []},
  68: {'ADR': [], 'DRUG': ['aspirin']},
  24: {'ADR': ['patent lad stent'], 'DRUG': []},
  33: {'ADR': ['aortic regurgitation'], 'DRUG': []},
  37: {'ADR': ['pericardial effusion'], 'DRUG': []},
  35: {'ADR': ['mitral valve prolapse'], 'DRUG': []},
  17: {'ADR': ['peptic ulcer disease'], 'DRUG': []},
  52: {'ADR': ['opacity left lung base'], 'DRUG': []},
  44: {'ADR': ['cyanosis edema'], 'DRUG': []},
  66: {'ADR': [], 'DRUG': ['atenolol']},
  39: {'ADR': ['eomi', 'sclera anicteric', 'mmm'], 'DRUG': []},
  93: {'ADR': [], 'DRUG': ['atenolol']},
  86: {'ADR': ['allergy'], 'DRUG': ['amoxicillin']},
  5: {'ADR': ['diverticulitis appendicitis'], 'DRUG': []},
  69: {'ADR': ['hyperlipidemia'], 'DRUG': ['pravastatin']},
  88: {'ADR': ['headache'], 'DRUG': []},
  64: {'ADR': ['hypotension'], 'DRUG': []},
  13: {'ADR': [

In [7]:
data = load_csv()

In [8]:
entries = [load_entry(data,i) for i in range(len(data))]

,type,name,value
0,T1,Drug 19078 19085,Aspirin
1,T2,Strength 19086 19092,325 mg
2,T3,Route 19093 19095,PO
3,T4,Drug 19465 19478;19479 19482,Ciprofloxacin HCl
4,T5,Strength 19483 19489,500 mg
...,...,...,...
168,R69,Reason-Drug Arg1:T102 Arg2:T59,NaN
169,R70,Reason-Drug Arg1:T103 Arg2:T16,NaN
170,R71,ADE-Drug Arg1:T94 Arg2:T8,NaN
171,T93,Route 12791 12793,IV


In [9]:
def check_truths(advs_pos, advs_pred):
    advs_pred_set = set(advs_pred)
    all_advs = [el[0] for el in advs_pos]
    correct_advs =  [adv for adv in all_advs if adv in advs_pred_set]
    target_advs = all_advs
    return correct_advs, target_advs


def get_from_pred(preds,type): # either ADR or DRUG
    return {adr for obj in preds.values() for adr in obj[type]}


def get_from_truth(annotated,type):
    sub_df = annotated[annotated['type'].str.startswith(
        'T') & annotated['name'].str.startswith(type)]
    return set(sub_df['value'].values)


In [34]:
def get_both(type1,type2):
    all_truths = [get_from_truth(entry.annotated_tsv,type1) for entry in entries]
    all_preds = [get_from_pred(groupedby_n2c2[entry.sample_id],type2) for entry in entries]
    return all_preds, all_truths

In [39]:
drug_preds, drug_truths = get_both('Drug','DRUG')
ade_preds , ade_truths = get_both('ADE','ADR')

In [45]:
def calc_recall(pred_set,truth_set):
    print(truth_set)
    if len(truth_set) == 0:
        return None
    return 1 - len(truth_set.difference(pred_set)) / len(truth_set)


In [46]:
import numpy as np
def calc_recall_avg(preds,truths):
    recs = [calc_recall(pred_set,truth_set) for pred_set,truth_set in zip(preds,truths)]
    filtered_recs = [rec for rec in recs if rec is not None]
    return np.mean(filtered_recs)


In [47]:

calc_recall_avg(drug_truths,drug_preds)

{'ativan', 'aspirin', 'ciprofloxacin', 'hadol', 'vicu', 'sma', 'tylenol'}
{'insulin', 'potassium phos', 'pantoprazole', 'motrin', 'ibuprofen'}
{'amitriptyline', 'coccaine', 'prednisone', 'omeprazole', 'dexamethasone', 'diphenhydramine', 'triamcinolone'}
{'lactulose', 'carvedilol', 'gabapentin', 'coumadin', 'oxycodone', 'levothyroxine', 'acetaminophen', 'zolpidem', 'bisacodyl', 'guaifenesin', 'lorazepam', 'docusate sodium', 'albuterol sulfate', 'aspirin', 'fluticasone', 'senna', 'lisinopril', 'lidocaine', 'vancomycin', 'statin', 'dextromethorphan', 'insulin glargine', 'afib', 'warfarin'}
{'simvastatin', 'troponin', 'latanoprost', 'levetiracetam', 'lisinopril', 'prn', 'pantoprazole', 'sodium 151', 'ferrous sulfate', 'levobunolol', 'sig'}
{'xanax', 'dulcolax', 'glyburide', 'aspirin', 'zantac', 'colace', 'thorazine', '- dur', 'elavil', 'penicillin haldol', 'lasix'}
{'efepime', 'coumadin', 'propofol', 'colchicine', 'isosorbide mononitrate', 'nimodipine', 'pravastatin', 'dilantin', 'zetia', 

0.3764507617091585

In [48]:
calc_recall_avg(drug_preds,drug_truths)

{'Metoprolol Tartrate', 'pain medication', 'Tylenol', 'Ativan', 'contrast', 'ativan', 'Acetaminophen-Caff- Butalbital', 'DiCYCLOmine', 'pain medications', 'butalbital -acetaminophen-caff', 'Aspirin EC', 'Ciprofloxacin HCl', 'Hadol', 'Omeprazole', 'GI cocktail', 'CONTRAST', 'Donnatol', 'Atorvastatin', 'PPI', 'ciprofloxacin [Cipro]', 'ATIVAN', 'Aspirin', 'stool softener', 'metoprolol tartrate'}
{'Insulin Glargine', 'NPH', 'Motrin', 'insulin', 'Potassium', 'Lancets', '70/30', 'IVF', 'Pantoprazol e', 'ibuprofe n', 'Humalog insulin', 'electrolytes (most notably K)', 'Insulin Lispro', 'antibiotics', 'Insulin', 'Regular ISS', 'Phos', 'insulin (Lantus)', 'pantoprazole', 'Reg'}
{'Fluocinonide', 'Gabapentin', 'DiphenhydrAMINE', 'dexamethasone', 'oxygen', 'Amitriptyline', 'Iodine', 'Biaxin', 'omeprazole', 'Sulfameth/Trimethoprim', 'amoxicillin', 'Triamcinolone Acetonide', 'Vicodin', 'Bactrim', 'Omeprazole', 'Loratadine', 'Symbicort *NF* (budesonide-formoterol)', 'prednisone', 'antifungal', 'Predn

0.1834595382488373

In [50]:

calc_recall_avg(ade_truths,ade_preds)

{'pelvic inguinal adenopathy', 'epigastric pain', 'hydronephrosis', 'extend celiac trunk', 'ulcerative atherosclerotic plaques', 'spleen', 'uterus ovaries', 'axillary , mediastinal , hilar lymphadenopathy', 'narrowing false lumen', 'effusions', 'mesenteric retroperitoneal lymphadenopathy', 'hemorrhagic', 'coldness', 'bleeding swelling', 'pancreas spleen', 'mass', 'abdominal pain', 'distal infrarenal aortic aneurysm level', 'foley', 'nodules', 'subclavian focal irregularity', 'compressive thrombus', 'supraclavicular , axillary , mediastinal hilar lymphadenopathy', 'hernia', 'diverticulosis', 'acute infarction', 'aortic aneurysm', 'mesenteric retroperitoneal adenopathy', 'bibasilar atelectasis', 'atherosclerotic calcification', 'fracture', 'large pleural effusion pneumothorax', 'pneumothorax', 'coronary artery calcifications', 'emphysema', 'bleeding stops', 'hemorrhage', 'centrilobular emphysema', 'mediastinal contours', 'bleeding stop', 'bilateral pleural effusions', 'numbness', 'corona

0.029756371152119068

In [49]:
calc_recall_avg(ade_preds,ade_truths)

{'delerious', 'acute delerium', 'very confused'}
{'hypoglycemia', 'sweating', 'palpitations', 'anxiety', 'irritated the lining of your stomach'}
set()
{'\\"wheezy', 'increased INR'}
set()
set()
set()
{'bradycardic', 'diarrhea', 'delirium', 'constipation', 'hypotension', 'Delirium'}
{'Diarrhea', 'Coagulopathy'}
{'mucositis', 'Odynophagia', 'pancytopenic'}
{'rash'}
set()
{'renal failure', 'anaphylactic reaction'}
{'withdrawal', 'diahrrea', 'anxiety', 'oversedated', 'withdrawl'}
{'hyperglycemia'}
{'sedation', 'supratherapeutic PTT', 'RR< 12'}
{'hypotension', 'Supratherapeutic INR'}
{'erythematous skin rash', 'acute renal failure', 'nausea', 'diarrhea', 'neurotoxicity', 'mild chills', 'respiratory distress', 'respiratory failure', 'tachypnea', 'stridor', 'fatigue'}
{'nausea', 'anaphylaxis', 'diarrhea', 'pruritis', 'fever', 'anaphylactic exposure', 'vomiting', 'anaphylactic reaction', 'chills', 'facial erythema/edema'}
{'Hypothyroid'}
{'paranoid', 'rigidity', 'psychotic', 'allergic reaction

0.2687833146693906